In [1]:
import urllib2
from bs4 import BeautifulSoup
from urlparse import urljoin
import sqlite3
import re
ignore_words = set(['the', 'to', 'of', 'and', 'a', 'in', 'is', 'it'])

In [2]:
page = 'https://baike.baidu.com/item/%E5%A3%B0%E4%B9%90'
c= urllib2.urlopen(page)
content = c.read()
soup = BeautifulSoup(content)
links = soup('a')
for link in links:
    if 'href' in dict(link.attrs):
        url = urljoin(page, link['href'])
        if url.find(";") != -1:
            continue
        url = url.split('#')[0]

/usr/local/lib/python2.7/dist-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html5lib"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /usr/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "html5lib")

  markup_type=markup_type))


In [3]:
class Crawler:
    def __init__(self, dbname):
        self.con = sqlite3.connect(dbname)

    def __del__(self):
        self.con.close()

    def db_commit(self):
        self.con.commit()

    # 获取id ,如果不存在，加入数据库
    def get_entryid(self, table, field, value, createnew=True):
        cur = self.con.execute("select rowid from %s where %s='%s'" % (table, field, value))
        res = cur.fetchone()
        if res==None:
            cur = self.con.execute("insert into %s (%s) values ('%s')" % (table, field, value))
            return cur.lastrowid
        else:
            return res[0]

    # 建索引
    def add_to_index(self, url, soup):
        if self.is_indexed(url):
            return
        print 'Indexing ' + url
        text = self.get_text_only(soup)
        words = self.get_saperate_words(text)
        urlid = self.get_entryid('urllist', 'url', url)
        for i in range(len(words)):
            word = words[i]
            if word in ignore_words:continue
            wordid = self.get_entryid('wordlist', 'word', word)
            self.con.execute("insert into wordlocation(urlid,wordid,location) values (%d,%d,%d)" % (urlid,wordid,i))

    # 从html网页中获取文字
    def get_text_only(self, soup):
        v = soup.string
        if v == None:
            c = soup.contents
            result_text = ''
            for t in c:
                subtext = self.get_text_only(t)
                result_text += subtext + '\n'
            return result_text
        else:
            return v.strip()

    # 分词处理
    def get_saperate_words(self, text):
        splitter = re.compile('\\W*')
        return [s.lower() for s in splitter.split(text) if s != '']

    def is_indexed(self, url):
        u = self.con.execute("select rowid from urllist where url = '%s'" % url).fetchone()
        if u != None:
            v = self.con.execute("select * from wordlocation where urlid = '%s'" % url)
            if v!= None:
                return True
        return False

    def add_link_ref(self, urlFrom, urlTo, linkText):
        pass

    def crawl(self, pages, depth=2):
        for i in range(depth):
            new_pages = set()
            for page in pages:
                try:
                    c = urllib2.urlopen(page)
                except:
                    print "Could not open %s " % page
                    continue
                soup = BeautifulSoup(c.read())
                self.add_to_index(page, soup)
                links = soup('a')
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find(";") != -1:
                            continue
                        url = url.split("#")[0]
                        if url[0:4]=='http' and not self.is_indexed(url):
                            new_pages.add(url)
                        link_text = self.get_text_only(link)
                        self.add_link_ref(page, url, link_text)
                self.db_commit()
            pages = new_pages

    def create_index_tables(self):
        self.con.execute('create table if not exists urllist(url) ')
        self.con.execute('create table if not exists wordlist(word)')
        self.con.execute('create table if not exists wordlocation(urlid,wordid,location)')
        self.con.execute('create table if not exists link(fromid integer,toid integer)')
        self.con.execute('create table if not exists linkwords(wordid,linkid)')
        self.con.execute('create index if not exists wordidx on wordlist(word)')
        self.con.execute('create index if not exists urlidx on urllist(url)')
        self.con.execute('create index if not exists wordurlidx on wordlocation(wordid)')
        self.con.execute('create index if not exists urltoidx on link(toid)')
        self.con.execute('create index if not exists urlfromidx on link(fromid)')
        self.db_commit()


In [4]:
pages = ['https://baike.baidu.com/item/%E5%A3%B0%E4%B9%90','https://www.zhihu.com/topic/19550228/hot']
crawler = Crawler('searchindex.db')
# crawler.create_index_tables()
# crawler.crawl(pages)

In [8]:
class Searcher:
    def __init__(self, dbname):
        self.con = sqlite3.connect(dbname)
        
    def __del__(self):
        self.con.close()
      
    def get_match_rows(self, q):
        field_list = 'w0.urlid'
        table_list = ''
        clause_list = ''
        word_ids = []
        
        words = q.split(' ')
        table_number = 0
        for word in words:
            word_row = self.con.execute("select rowid from wordlist where word='%s'" % word).fetchone()
            if word_row is not None:
                word_id = word_row[0]
                word_ids.append(word_id)
                if table_number > 0:
                    table_list += ','
                    clause_list += ' and '
                    clause_list += 'w%d.urlid=w%d.urlid and ' % (table_number -1, table_number)
                field_list += ',w%d.location' % table_number
                table_list += 'wordlocation w%d' % table_number
                clause_list += 'w%d.wordid = %d' % (table_number, word_id)
        full_query = 'select %s from %s where %s' % (field_list, table_list, clause_list)
        cur = self.con.execute(full_query)
        rows = [row for wor in cur]
        return rows, word_ids
        